In [1]:
import fsspec
import zarr
import xarray as xr
from kerchunk.combine import MultiZarrToZarr

In [2]:
file = 's3://noaa-nodd-kerchunk-pds/nos/lmhofs/nos.lmhofs.fields.f000.20240520.t00z.nc.zarr'

reffs = fsspec.filesystem(
        "reference",
        fo=file,
    )

m = reffs.get_mapper("")

ds = xr.open_zarr(m, consolidated=False, drop_variables=['Itime', 'Itime2'])
ds


/Users/matthew.iannucci/Developer/ocean-notebooks/venv/lib/python3.10/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:             (time: 1, four: 4, nele: 171377, node: 90806, three: 3,
                         siglev: 21, maxnode: 11, maxelem: 9, siglay: 20)
Coordinates:
    lat                 (node) float32 dask.array<chunksize=(90806,), meta=np.ndarray>
    latc                (nele) float32 dask.array<chunksize=(171377,), meta=np.ndarray>
    lon                 (node) float32 dask.array<chunksize=(90806,), meta=np.ndarray>
    lonc                (nele) float32 dask.array<chunksize=(171377,), meta=np.ndarray>
    siglay              (siglay, node) float32 dask.array<chunksize=(20, 90806), meta=np.ndarray>
    siglev              (siglev, node) float32 dask.array<chunksize=(21, 90806), meta=np.ndarray>
  * time                (time) datetime64[ns] 2024-05-20
Dimensions without coordinates: four, nele, node, three, maxnode, maxelem
Data variables: (12/49)
    Times               (time) |S26 dask.array<chunksize=(1,), meta=np.ndarray>
    a1u                 (four, nele) float32 dask.array<chunksize=(4, 171377), meta=np.ndarray>
    a2u                 (four, nele) float32 dask.array<chunksize=(4, 171377), meta=np.ndarray>
    art1                (node) float32 dask.array<chunksize=(90806,), meta=np.ndarray>
    art2                (node) float32 dask.array<chunksize=(90806,), meta=np.ndarray>
    atmos_press         (time, node) float32 dask.array<chunksize=(1, 90806), meta=np.ndarray>
    ...                  ...
    ww                  (time, siglay, nele) float32 dask.array<chunksize=(1, 20, 171377), meta=np.ndarray>
    x                   (node) float32 dask.array<chunksize=(90806,), meta=np.ndarray>
    xc                  (nele) float32 dask.array<chunksize=(171377,), meta=np.ndarray>
    y                   (node) float32 dask.array<chunksize=(90806,), meta=np.ndarray>
    yc                  (nele) float32 dask.array<chunksize=(171377,), meta=np.ndarray>
    zeta                (time, node) float32 dask.array<chunksize=(1, 90806), meta=np.ndarray>
Attributes: (12/14)
    Conventions:                 CF-1.0
    CoordinateProjection:        proj=utm +zone=16 +datum=NAD83
    CoordinateSystem:            GeoReferenced
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    River_Forcing:               THERE ARE 12 RIVERS IN THIS MODEL.\nRIVER IN...
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    ...                          ...
    Tidal_Forcing:               Tidal Forcing Time Series Title: lmhofs late...
    history:                     model started at: 20/05/2024   02:02
    institution:                 School for Marine Science and Technology
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    source:                      FVCOM_4.3
    title:                       LMHOFS

In [3]:
files = [
    's3://noaa-nodd-kerchunk-pds/nos/lmhofs/nos.lmhofs.fields.f001.20240520.t00z.nc.zarr',
    's3://noaa-nodd-kerchunk-pds/nos/lmhofs/nos.lmhofs.fields.f002.20240520.t00z.nc.zarr',
    's3://noaa-nodd-kerchunk-pds/nos/lmhofs/nos.lmhofs.fields.f003.20240520.t00z.nc.zarr',
]

mzz = MultiZarrToZarr(
    files, 
    remote_protocol='s3', 
    remote_options={'anon': True},
    concat_dims=['time'],
    identical_dims=[
        'lat',
        'lon',
        'nv',
        'latc',
        'lonc',
    ]
)

mzz.translate()

{'version': 1,
 'refs': {'.zgroup': '{"zarr_format":2}',
  'time/.zarray': '{\n    "chunks": [\n        3\n    ],\n    "compressor": null,\n    "dtype": "<f4",\n    "fill_value": null,\n    "filters": null,\n    "order": "C",\n    "shape": [\n        3\n    ],\n    "zarr_format": 2\n}',
  'time/0': 'base64:q7ARRVWxEUUAshFF',
  'time/.zattrs': '{\n    "_ARRAY_DIMENSIONS": [\n        "time"\n    ],\n    "format": "defined reference date",\n    "long_name": "time",\n    "time_zone": "UTC",\n    "units": "days since 2018-01-01 00:00:00"\n}',
  '.zattrs': '{"Conventions":"CF-1.0","CoordinateProjection":"proj=utm +zone=16 +datum=NAD83","CoordinateSystem":"GeoReferenced","GroundWater_Forcing":"GROUND WATER FORCING IS OFF!","River_Forcing":"THERE ARE 12 RIVERS IN THIS MODEL.\\nRIVER INFLOW IS ON THE edges WHERE TEMPERATURE AND SALINITY ARE specified IN THE MODEL.\\nTHE FOLLOWING RIVER NAMES ARE USED:\\n04127885\\n04127885\\n04159130\\n04159130\\n04159130\\n04157005\\n04157005\\n04157005\\n0415